In [1]:
import nltk
word_list = nltk.word_tokenize("I love NLP")
print(word_list)
print(nltk.pos_tag(word_list))


['I', 'love', 'NLP']
[('I', 'PRP'), ('love', 'VBP'), ('NLP', 'RB')]


In [2]:
import jieba

# 1.分词
seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Precise Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print("Default Mode: " + ", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))   # 此模式适合用于搜索引擎构建倒排索引的分词，粒度比较细

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/fv/xhx25_714xx6h8fhfqc7v45h0000gn/T/jieba.cache
Loading model cost 0.879 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Precise Mode: 我/ 来到/ 北京/ 清华大学
Default Mode: 他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


In [3]:
import collections

# 字符统计
text = "我爱自然语言处理。自然语言处理是一个很有意思的研究领域。"
c = collections.Counter(text)
print(c)

# 词频统计
import jieba
seg_list = jieba.cut("我爱自然语言处理。自然语言处理是一个很有意思的研究领域。", cut_all=False)
c = collections.Counter(list(seg_list))
print(c)


Counter({'自': 2, '然': 2, '语': 2, '言': 2, '处': 2, '理': 2, '。': 2, '我': 1, '爱': 1, '是': 1, '一': 1, '个': 1, '很': 1, '有': 1, '意': 1, '思': 1, '的': 1, '研': 1, '究': 1, '领': 1, '域': 1})
Counter({'自然语言': 2, '处理': 2, '。': 2, '我': 1, '爱': 1, '是': 1, '一个': 1, '很': 1, '有意思': 1, '的': 1, '研究': 1, '领域': 1})


In [4]:
# 1.基于sklearn的N-gram
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import jieba

data = ["他用报话机向上级呼喊：“为了祖国，为了胜利，向我开炮！向我开炮！",
        "记者：你怎么会说出那番话？",
        "韦昌进：我只是觉得，对准我自己打，才有可能把上了我哨位的这些敌人打死，或者打下去。"]

# data = [" ".join(jieba.lcut(e)) for e in data] # jieba分词，并用" "连接

vec = CountVectorizer(min_df=1, ngram_range=(2, 2))
# ngram_range=(1,1) 表示 unigram, ngram_range=(2,2) 表示 bigram, ngram_range=(3,3) 表示 thirgram
# min_df 为阈值，如果某个词的文本频率小于min_df，则这个词不会被当作关键词

#vec = CountVectorizer(ngram_range=(2, 4), decode_error="ignore", token_pattern = r'\b\w+\b',min_df=1
#如果ngram_range=(2, 4)，则表示2，3,4个单词切割

# 最后用fit()和transform()进行拟合和计算
X = vec.fit_transform(data)  # transform text to metrix
print(X)
vec.get_feature_names() # get features

X.toarray()
# 查看生成的词表
print(vec.vocabulary_)

df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names()) # to DataFrame
df.head()

  (0, 3)	1
  (0, 1)	1
  (0, 0)	1
  (0, 2)	1
  (1, 7)	1
  (2, 6)	1
  (2, 4)	1
  (2, 5)	1
  (2, 8)	1
{'他用报话机向上级呼喊 为了祖国': 2, '为了祖国 为了胜利': 0, '为了胜利 向我开炮': 1, '向我开炮 向我开炮': 3, '记者 你怎么会说出那番话': 7, '韦昌进 我只是觉得': 8, '我只是觉得 对准我自己打': 5, '对准我自己打 才有可能把上了我哨位的这些敌人打死': 4, '才有可能把上了我哨位的这些敌人打死 或者打下去': 6}


,为了祖国 为了胜利,为了胜利 向我开炮,他用报话机向上级呼喊 为了祖国,向我开炮 向我开炮,对准我自己打 才有可能把上了我哨位的这些敌人打死,我只是觉得 对准我自己打,才有可能把上了我哨位的这些敌人打死 或者打下去,记者 你怎么会说出那番话,韦昌进 我只是觉得
0,1,1,1,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,1,1,1,0,1


In [8]:
# 2.基于jieba的N-gram

# 这里的_word_ngrams方法其实就是sklearn中CountVectorizer函数中用于N-Gram的方法
def _word_ngrams(tokens, stop_words=None, ngram_range=(1,1)):
        """Turn tokens into a sequence of n-grams after stop words filtering"""
        # handle stop words
        if stop_words is not None:
            tokens = [w for w in tokens if w not in stop_words]

        # handle token n-grams
        min_n, max_n = ngram_range
        if max_n != 1:
            original_tokens = tokens
            tokens = []
            n_original_tokens = len(original_tokens)
            for n in range(min_n, min(max_n + 1, n_original_tokens + 1)):
                for i in range(n_original_tokens - n + 1):
                    tokens.append(" ".join(original_tokens[i: i + n]))

        return tokens

text = "我去云南旅游，不仅去了玉龙雪山，还去丽江古城，很喜欢丽江古城"
import jieba
cut = jieba.cut(text)
listcut = list(cut)
n_gramWords = _word_ngrams(tokens = listcut, ngram_range=(2,2))
print(n_gramWords)
for n_gramWord in n_gramWords:
    print(n_gramWord)


['我 去', '去 云南旅游', '云南旅游 ，', '， 不仅', '不仅 去', '去 了', '了 玉龙雪山', '玉龙雪山 ，', '， 还', '还 去', '去 丽江', '丽江 古城', '古城 ，', '， 很', '很 喜欢', '喜欢 丽江', '丽江 古城']
我 去
去 云南旅游
云南旅游 ，
， 不仅
不仅 去
去 了
了 玉龙雪山
玉龙雪山 ，
， 还
还 去
去 丽江
丽江 古城
古城 ，
， 很
很 喜欢
喜欢 丽江
丽江 古城


In [ ]:
# 3.基于NLTK的N-gram
import nltk

list(nltk.bigrams(['a','b','c']))


In [17]:
## 文本矩阵化

#CountVectorizer 类会将文本中的词语转换为词频矩阵，例如矩阵中包含一个元素a[i][j]，它表示j词在i类文本下的词频。它通过 fit_transform 函数计算各个词语出现的次数，通过get_feature_names()可获取词袋中所有文本的关键字，通过 toarray()可看到词频矩阵的结果。

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=1)

corpus = ['This is the first document.', 'This is the second document.', 'And the third one.' 'Is this the first document?']

X = vectorizer.fit_transform(corpus)
feature_name = vectorizer.get_feature_names()

print(X)   #左边的括号中的第一个数字是文本的序号i，第2个数字是词的序号j，注意词的序号是基于所有的文档的。第三个数字就是我们的词频。
print(feature_name)
print(X.toarray())
# 由于大部分文本都只会用词汇表中很少一部分的词，因此词向量中有大量的0，也就是说词向量是稀疏的。因此在实际应用中一般使用稀疏矩阵来存储。



# 因此，有些词在文本中尽管词频高，但是并不重要，这个时候就可以用TF-IDF技术
# 1. CountVectorizer 结合 TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer()
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus))
print(tfidf)

# 2.用 TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer()
re = tfidf2.fit_transform(corpus)
print(re)



## jieba的TF-IDF

  (0, 1)	1
  (0, 2)	1
  (0, 6)	1
  (0, 3)	1
  (0, 8)	1
  (1, 5)	1
  (1, 1)	1
  (1, 6)	1
  (1, 3)	1
  (1, 8)	1
  (2, 4)	1
  (2, 7)	1
  (2, 0)	1
  (2, 1)	1
  (2, 2)	1
  (2, 6)	2
  (2, 3)	1
  (2, 8)	1
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 1 1 0 1]
 [1 1 1 1 1 0 2 1 1]]
  (0, 8)	0.42040098658605557
  (0, 6)	0.42040098658605557
  (0, 3)	0.42040098658605557
  (0, 2)	0.5413428136679054
  (0, 1)	0.42040098658605557
  (1, 8)	0.3816141458138271
  (1, 6)	0.3816141458138271
  (1, 5)	0.6461289150464732
  (1, 3)	0.3816141458138271
  (1, 1)	0.3816141458138271
  (2, 8)	0.2407133333247863
  (2, 7)	0.4075631016420483
  (2, 6)	0.4814266666495726
  (2, 4)	0.4075631016420483
  (2, 3)	0.2407133333247863
  (2, 2)	0.30996224392243715
  (2, 1)	0.2407133333247863
  (2, 0)	0.4075631016420483
  (0, 8)	0.42040098658605557
  (0, 3)	0.42040098658605557
  (0, 6)	0.42040098658605557
  (0, 2)	0.5413428136679054
  (0, 1)	0.42040098658605557
  (1, 8)	